In [234]:
import pandas as pd
import glob
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import random
from collections import defaultdict

In [235]:
movies = []
for summary_path in glob.glob('data/movies/*/summary.json'):
    imdb_id = summary_path.split('/')[2].split('_')[0]
    with open(summary_path, 'r') as fp:
        movie = json.load(fp)
        movie['id'] = imdb_id
        movies.append(movie)
movies_df = pd.DataFrame(movies)

In [203]:
movies_df.head()

,title,genres,director,starring,rating,release_date,runtime,certificate,img_url,plot,id
0,The Hangover,Comedy,Todd Phillips,"Zach Galifianakis, Bradley Cooper, Justin Bartha",7.7,"June 5, 2009 (United States)",1 hour 40 minutes,R,https://m.media-amazon.com/images/M/MV5BNGQwZj...,Three buddies wake up from a bachelor party in...,tt1119646
1,Ladder 49,"Action, Drama, Thriller",Jay Russell,"Joaquin Phoenix, John Travolta, Jacinda Barrett",6.5,"October 1, 2004 (United States)",1 hour 55 minutes,PG-13,https://m.media-amazon.com/images/M/MV5BMjEyND...,"A firefighter, injured and trapped in a burnin...",tt0349710
2,Mark of the Vampire,"Horror, Mystery",Tod Browning,"Lionel Barrymore, Elizabeth Allan, Bela Lugosi",6.3,"April 26, 1935 (United States)",1 hour,Passed,https://m.media-amazon.com/images/M/MV5BMTY4MT...,"When a nobleman is murdered, a professor of th...",tt0026685
3,The Mark of Zorro,"Action, Adventure, Family",Rouben Mamoulian,"Tyrone Power, Linda Darnell, Basil Rathbone",7.5,"November 8, 1940 (United States)",1 hour 34 minutes,Approved,https://m.media-amazon.com/images/M/MV5BMTU2OD...,A young Spanish aristocrat must masquerade as ...,tt0032762
4,Son of Paleface,"Comedy, Western",Frank Tashlin,"Bob Hope, Jane Russell, Roy Rogers",6.8,"July 14, 1952 (United States)",1 hour 35 minutes,Passed,https://m.media-amazon.com/images/M/MV5BMzc1Nz...,Junior Potter returns to claim his father's go...,tt0045177


In [214]:
movies_df['release_year'] = pd.to_datetime(movies_df.release_date.str.split('(').apply(lambda x: x[0]), format='mixed').dt.year

In [220]:
movies_df['release_year'].value_counts().sort_index(ascending=False)[:20]

release_year
2023     56
2022    244
2021    257
2020    214
2019    286
2018    307
2017    289
2016    269
2015    287
2014    288
2013    279
2012    260
2011    278
2010    248
2009    261
2008    271
2007    273
2006    251
2005    230
2004    199
Name: count, dtype: int64

In [231]:
movies_df[movies_df['title'] == 'Parasite']

,title,genres,director,starring,rating,release_date,runtime,certificate,img_url,plot,id,release_year
2582,Parasite,"Drama, Thriller",Bong Joon Ho,"Song Kang-ho, Lee Sun-kyun, Cho Yeo-jeong",8.5,"November 8, 2019 (United States)",2 hours 12 minutes,R,https://m.media-amazon.com/images/M/MV5BYWZjMj...,Greed and class discrimination threaten the ne...,tt6751668,2019


In [232]:
movies_df[movies_df['release_year'] == 2019].sort_values('rating', ascending=False)[:20]

,title,genres,director,starring,rating,release_date,runtime,certificate,img_url,plot,id,release_year
2582,Parasite,"Drama, Thriller",Bong Joon Ho,"Song Kang-ho, Lee Sun-kyun, Cho Yeo-jeong",8.5,"November 8, 2019 (United States)",2 hours 12 minutes,R,https://m.media-amazon.com/images/M/MV5BYWZjMj...,Greed and class discrimination threaten the ne...,tt6751668,2019
3713,Joker,"Crime, Drama, Thriller",Todd Phillips,"Joaquin Phoenix, Robert De Niro, Zazie Beetz",8.4,"October 4, 2019 (United States)",2 hours 2 minutes,R,https://m.media-amazon.com/images/M/MV5BNGVjNW...,"The rise of Arthur Fleck, from aspiring stand-...",tt7286456,2019
4463,Kaithi,"Action, Adventure, Crime",Lokesh Kanagaraj,"Karthi, Narain, Arjun Das",8.4,"October 24, 2019 (United States)",2 hours 25 minutes,Not Rated,https://m.media-amazon.com/images/M/MV5BZjQyNT...,"Dilli, an ex-convict, endeavours to meet his d...",tt9900782,2019
6993,Avengers: Endgame,"Action, Adventure, Drama",Anthony Russo,"Robert Downey Jr., Chris Evans, Mark Ruffalo",8.4,"April 26, 2019 (United States)",3 hours 1 minute,PG-13,https://m.media-amazon.com/images/M/MV5BMTc5MD...,After the devastating events of Avengers: Infi...,tt4154796,2019
8342,Asuran,"Action, Drama",Vetrimaaran,"Dhanush, Manju Warrier, Prakash Raj",8.4,"October 3, 2019 (United States)",2 hours 21 minutes,Not Rated,https://m.media-amazon.com/images/M/MV5BZTgzNT...,The teenage son of a farmer from an underprivi...,tt9477520,2019
9801,Chhichhore,"Comedy, Drama, Romance",Nitesh Tiwari,"Sushant Singh Rajput, Shraddha Kapoor, Varun S...",8.3,"September 6, 2019 (United States)",2 hours 23 minutes,Not Rated,https://m.media-amazon.com/images/M/MV5BYjg2ZD...,"A tragic incident forces Anirudh, a middle-age...",tt9052870,2019
8563,Miracle in Cell No. 7,Drama,Mehmet Ada Öztekin,"Aras Bulut Iynemli, Nisa Sofiya Aksongur, Deni...",8.2,"October 11, 2019 (Turkey)",2 hours 12 minutes,TV-14,https://m.media-amazon.com/images/M/MV5BOGE3N2...,A story of love between a mentally-ill father ...,tt10431500,2019
4431,Uri: The Surgical Strike,"Action, Drama, History",Aditya Dhar,"Vicky Kaushal, Paresh Rawal, Mohit Raina",8.2,"January 11, 2019 (United States)",2 hours 18 minutes,Not Rated,https://m.media-amazon.com/images/M/MV5BMWU4Zj...,Indian army special forces execute a covert op...,tt8291224,2019
10781,Klaus,"Animation, Adventure, Comedy",Sergio Pablos,"Jason Schwartzman, J.K. Simmons, Rashida Jones",8.2,"November 15, 2019 (United States)",1 hour 36 minutes,PG,https://m.media-amazon.com/images/M/MV5BMWYwOT...,A simple act of kindness always sparks another...,tt4729430,2019
3543,Ford v Ferrari,"Action, Biography, Drama",James Mangold,"Matt Damon, Christian Bale, Jon Bernthal",8.1,"November 15, 2019 (United States)",2 hours 32 minutes,PG-13,https://m.media-amazon.com/images/M/MV5BM2UwMD...,American car designer Carroll Shelby and drive...,tt1950186,2019


## Experiment 1: Taking the first embedding only

In [236]:
# take the first summary embedding only
embeddings = dict()
for embedding_path in glob.glob('data/embeddings_v1/*/0.txt'):
    movie_id = embedding_path.split('/')[2].split('_')[0]
    with open(embedding_path, 'r') as fp:
        embedding = [float(val) for val in fp.read().split(',')]
    embeddings[movie_id] = embedding

In [ ]:
with open('data/embeddings_v1')
embeddings.to

In [159]:
embedding_matrix = list()
movie_ids = list()
for movie_id, embedding in embeddings.items():
    embedding_matrix.append(embedding)
    movie_ids.append(movie_id)
embedding_matrix = np.array(embedding_matrix)
similarities_v1 = cosine_similarity(embedding_matrix)

In [160]:
filtered_movies_df = movies_df[movies_df['id'].isin(movie_ids)].reset_index(drop=True).copy()

In [172]:
# pick a movie at random
# random_index = int(random.random() * filtered_movies_df.shape[0])
random_index = 2960
print(f"Random Movie: {filtered_movies_df.iloc[random_index].title}. Genre: {filtered_movies_df.iloc[random_index].genres}")
print(filtered_movies_df.iloc[random_index]['plot'])
print('\n')
# rank the movie similarities
movie_similarities = [(movie_ids[movie_id], similarity) for movie_id, similarity in enumerate(similarities_v1[random_index])]
movie_similarities = sorted(movie_similarities, key=lambda x: x[1])[::-1]
most_similar_movie_id = movie_similarities[1][0]
most_different_movie_id = movie_similarities[-1][0]

most_similar_movie = filtered_movies_df[filtered_movies_df['id'] == most_similar_movie_id].iloc[0]
most_different_movie = filtered_movies_df[filtered_movies_df['id'] == most_different_movie_id].iloc[0]

print(f"Most Similar Movies:")
for movie_id, _ in movie_similarities[1:5]:
    movie = filtered_movies_df[filtered_movies_df['id'] == movie_id].iloc[0]
    print(f"{movie.title}. Genre: {movie.genres}")
    print(movie['plot'])
    print('\n')
print(f"Most Different Movie: {most_different_movie.title}. Genre: {most_different_movie.genres}")
print(most_different_movie['plot'])

Random Movie: Inception. Genre: Action, Adventure, Sci-Fi
A thief who steals corporate secrets through the use of dream-sharing technology is given the inverse task of planting an idea into the mind of a C.E.O., but his tragic past may doom the pro... Read all


Most Similar Movies:
Swordfish. Genre: Action, Crime, Thriller
A covert counter-terrorist unit called Black Cell led by Gabriel Shear wants the money to help finance their war against international terrorism, but it's all locked away. Gabriel brings in ... Read all


The Collector. Genre: Action, Horror, Thriller
Desperate to repay his debt to his ex-wife, an ex-con plots a heist at his new employer's country home, unaware that a second criminal has also targeted the property, and rigged it with a se... Read all


Possessor. Genre: Horror, Mystery, Sci-Fi
An agent works for a secretive organization that uses brain-implant technology to inhabit other people's bodies - ultimately driving them to commit assassinations for high-pay

## Experiment 2: Average of all summary embeddings

In [175]:
# take all the summary embeddings
embeddings = defaultdict(list)
for embedding_path in glob.glob('data/embeddings_v1/*/*.txt'):
    movie_id = embedding_path.split('/')[2].split('_')[0]
    with open(embedding_path, 'r') as fp:
        embedding = [float(val) for val in fp.read().split(',')]
    embeddings[movie_id].append(embedding)

In [179]:
embeddings[list(embeddings.keys())[0]]

[[-0.0003313207416795194,
  -0.030928298830986023,
  -0.0007028778200037777,
  -0.03409355506300926,
  -0.007148648612201214,
  0.01710042543709278,
  -0.003379848785698414,
  -0.005391663871705532,
  -0.01769055798649788,
  -0.02222384698688984,
  -0.015397089533507824,
  0.025375690311193466,
  0.006632282864302397,
  0.020936286076903343,
  0.030123572796583176,
  -0.010012132115662098,
  0.027870340272784233,
  -0.004607055801898241,
  0.017113838344812393,
  -0.021338649094104767,
  -0.015651918947696686,
  -0.009368350729346275,
  -0.012419603765010834,
  0.006142741069197655,
  0.0006727005820721388,
  0.004969182424247265,
  0.018830586224794388,
  0.0002871027390938252,
  0.014713072218000889,
  -0.0031099303159862757,
  0.014726484194397926,
  -0.013505983166396618,
  -0.020842401310801506,
  -0.0231492817401886,
  -0.02693149447441101,
  -0.04265047237277031,
  0.01038096472620964,
  -0.009824362583458424,
  0.035300642251968384,
  0.0016823800979182124,
  0.0083087952807545

In [184]:
# now average them
embedding_matrix = list()
movie_ids = list()
for movie_id, embedding in embeddings.items():
    embedding_matrix.append(np.mean(np.array(embedding), axis=0))
    movie_ids.append(movie_id)
embedding_matrix = np.array(embedding_matrix)
similarities_v2 = cosine_similarity(embedding_matrix)

In [185]:
embedding_matrix

array([[-0.00124343, -0.02742286,  0.00123543, ..., -0.0240431 ,
         0.00038762, -0.01312255],
       [ 0.0024492 , -0.03230757, -0.00805298, ..., -0.01764698,
        -0.02448932, -0.00187325],
       [ 0.000512  , -0.01278176,  0.01290342, ..., -0.02151763,
         0.0037973 , -0.01818782],
       ...,
       [ 0.02227753, -0.02493832, -0.01102957, ..., -0.00450227,
         0.00220199, -0.00501869],
       [ 0.00547939, -0.01667477,  0.00557582, ..., -0.01826225,
        -0.0116086 , -0.00330755],
       [ 0.01236798, -0.01456697, -0.01989475, ..., -0.0149271 ,
         0.00092609, -0.02832427]])

In [187]:
filtered_movies_df = movies_df[movies_df['id'].isin(movie_ids)].reset_index(drop=True).copy()

In [195]:
filtered_movies_df[filtered_movies_df.title == 'Inception']

,title,genres,director,starring,rating,release_date,runtime,certificate,img_url,plot,id
2960,Inception,"Action, Adventure, Sci-Fi",Christopher Nolan,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellio...",8.8,"July 16, 2010 (United States)",2 hours 28 minutes,PG-13,https://m.media-amazon.com/images/M/MV5BMjAxMz...,A thief who steals corporate secrets through t...,tt1375666


In [192]:
filtered_movies_df.shape

(3866, 11)

In [200]:
# pick a movie at random
random_index = int(random.random() * filtered_movies_df.shape[0])
print(f"Random Movie: {filtered_movies_df.iloc[random_index].title}. Genre: {filtered_movies_df.iloc[random_index].genres}")
print(filtered_movies_df.iloc[random_index]['plot'])
print('\n')
# rank the movie similarities
movie_similarities = [(movie_ids[movie_id], similarity) for movie_id, similarity in enumerate(similarities_v2[random_index])]
movie_similarities = sorted(movie_similarities, key=lambda x: x[1])[::-1]
most_similar_movie_id = movie_similarities[1][0]
most_different_movie_id = movie_similarities[-1][0]

most_similar_movie = filtered_movies_df[filtered_movies_df['id'] == most_similar_movie_id].iloc[0]
most_different_movie = filtered_movies_df[filtered_movies_df['id'] == most_different_movie_id].iloc[0]

print(f"Most Similar Movies:")
for movie_id, _ in movie_similarities[1:5]:
    movie = filtered_movies_df[filtered_movies_df['id'] == movie_id].iloc[0]
    print(f"{movie.title}. Genre: {movie.genres}")
    print(movie['plot'])
    print('\n')
print(f"Most Different Movie: {most_different_movie.title}. Genre: {most_different_movie.genres}")
print(most_different_movie['plot'])

Random Movie: Le Cercle Rouge. Genre: Crime, Drama, Thriller
After leaving prison, master thief Corey crosses paths with a notorious escapee and an alcoholic former policeman. The trio proceed to plot an elaborate heist.


Most Similar Movies:
Reservoir Dogs. Genre: Crime, Thriller
When a simple jewelry heist goes horribly wrong, the surviving criminals begin to suspect that one of them is a police informant.


Thief. Genre: Action, Crime, Drama
An ace safe cracker wants to do one last big heist for the mob before going straight.


The Asphalt Jungle. Genre: Crime, Drama, Film-Noir
A major heist goes off as planned, but then double crosses, bad luck and solid police work cause everything to unravel.


Rififi. Genre: Crime, Drama, Thriller
Four men plan a technically perfect crime, but the human element intervenes...


Most Different Movie: The Professor and the Madman. Genre: Biography, Drama, History
Professor James Murray begins work compiling words for the first edition of the Oxfor